# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
# import libraries

import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB

from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import FeatureUnion, Pipeline


In [3]:
import os
os.path.abspath(os.getcwd())

'/home/workspace'

### Loading Up DATABASE 'disaster_response' Prepared from ETL Stage

In [4]:
# load data from database
#def load_data(data_file)
def load_data():
    engine = create_engine('sqlite:///disaster_response.db')
    conn = engine.connect()
    df = pd.read_sql_table('disaster_response', conn)
    #df.head(2)
    X = df.message.values
    
    y = df.iloc[:, 4:]
    ##y.dropna(axis = 0, how = 'any', inplace=True)
    ##y.fillna(0, inplace=True)
    colnames = y.columns
    #y = y.values
    engine.dispose()
    
    return X, y, colnames

### Starting Verb Extractor

Starting Verb Extractor Function is creating a Few NAN values, that is why avoiding its use for now

In [8]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        # tokenize by sentences
        sentence_list = nltk.sent_tokenize(text)
        
        for sentence in sentence_list:
            # tokenize each sentence into words and tag part of speech
            pos_tags = nltk.pos_tag(word_tokenize(sentence))

            # index pos_tags to get the first word and part of speech tag
            first_word, first_tag = pos_tags[0][0], pos_tags[0][1]
            
            # return true if the first word is an appropriate verb or RT for retweet
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return 1

            return 0

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        # apply starting_verb function to all values in X
        X_tagged = pd.Series(X).apply(self.starting_verb)

        return pd.DataFrame(X_tagged)

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    
    tokens = word_tokenize(text)
    tokens_wihtout_sw = [w for w in tokens if w not in stopwords.words("english") ]
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    
    for tok in tokens_wihtout_sw:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

        
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

** Feature Union Enables Parallel Execution of Code - Removed

In [6]:
def model_pipeline():
    pipeline = Pipeline([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)), 
            ('tfidf', TfidfTransformer())
        ])),
        ('mclf', MultiOutputClassifier(RandomForestClassifier()))
#        ('clf', RandomForestClassifier())
    ])
    
    return pipeline

model-pipeline_with_sw to be used when using verb extracting function

#### 3.2 With GridSearch

In [7]:
def model_pipeline_with_GS():
    pipeline = Pipeline([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)), 
            ('tfidf', TfidfTransformer())
        ])),
        ('mclf', MultiOutputClassifier(RandomForestClassifier()))
#        ('clf', RandomForestClassifier())
    ])
    
    parameters = {
        #'text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        #'text_pipeline__vect__max_df': (0.5, 0.75),
        #'text_pipeline__vect__max_features': (None, 7000),
        #'text_pipeline__tfidf__use_idf': (True, False),
        #'mclf__estimator__max_depth': [2, 3],
        #'mclf__estimator__min_samples_split': [2, 3],
        'mclf__estimator__n_estimators': [50, 70],
        #'mclf__estimator__max_leaf_nodes' : [3,4]
        
        
    }

    #cv = RandomizedSearchCV(pipeline, param_distributions=parameters)
    cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs = 4, cv = 2, verbose = 3)
    #print(cv.best_params_)
    
    return cv

In [8]:
m = MultiOutputClassifier(LogisticRegression())
m.get_params().keys()

dict_keys(['estimator__C', 'estimator__class_weight', 'estimator__dual', 'estimator__fit_intercept', 'estimator__intercept_scaling', 'estimator__max_iter', 'estimator__multi_class', 'estimator__n_jobs', 'estimator__penalty', 'estimator__random_state', 'estimator__solver', 'estimator__tol', 'estimator__verbose', 'estimator__warm_start', 'estimator', 'n_jobs'])

#### 3.3 With Multiple Models

In [9]:
from sklearn.base import BaseEstimator
class ClfSwitcher(BaseEstimator):

    def __init__(
        self, 
        estimator = MultinomialNB(),
    ):
#    """
#    A Custom BaseEstimator that can switch between classifiers.
#    :param estimator: sklearn object - The classifier
#    """ 
      #return self.estimator 
    
        self.estimator = estimator
        
    def fit(self, X, y): #pass
        return self.estimator.fit(X,y)
        #pass
    
    def predict(self, X, y=None):
        return self.estimator.predict(X)

    
    def score(self, X, y): #pass
        return self.estimator.score(X, y)
        #pass

In [10]:
def model_pipeline_with_Multiple():
    pipeline = Pipeline([
        ('text_pipeline',  Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)), 
            ('tfidf', TfidfTransformer())
        ])),
        ('clf', ClfSwitcher())
    ])
    

    parameters = [
        {
            'clf__estimator': [MultiOutputClassifier(DecisionTreeClassifier())], # SVM if hinge loss / logreg if log loss
            #'tfidf__max_df': (0.25, 0.5),
            #'tfidf__stop_words': ['english', None],
            #'clf__estimator__penalty': ('l2', 'elasticnet', 'l1'),
            #'clf__estimator__max_iter': [50, 80],
            #'clf__estimator__tol': [1e-4],
            #'clf__estimator__loss': ['hinge', 'log', 'modified_huber'],
        },
        #{
        #    'clf__estimator': [RandomForestClassifier()], # SVM if hinge loss / logreg if log loss
            #'tfidf__max_df': (0.25, 0.5),
            #'tfidf__stop_words': ['english', None],
            #'clf__estimator__penalty': ('l2', 'elasticnet', 'l1'),
            #'clf__estimator__max_iter': [50, 80],
            #'clf__estimator__tol': [1e-4],
            #'clf__estimator__loss': ['hinge', 'log', 'modified_huber'],
        #},
        #{
        #    'clf__estimator': [MultiOutputClassifier(LogisticRegression())],
            #'text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
            #'text_pipeline__vect__max_df': (0.5, 0.75),
            #'text_pipeline__vect__max_features': (None, 7000),
            #'text_pipeline__tfidf__use_idf': (True, False),
            #'clf__estimator__max_depth': [2, 3],
            #'clf__estimator__min_samples_split': [2, 3],
            #'clf__estimator__n_estimators': [50, 70],
            #'clf__estimator__max_leaf_nodes' : [3,4]
        #},
        {
            'clf__estimator': [MultiOutputClassifier(MLPClassifier())],
            #'text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
            #'text_pipeline__vect__max_df': (0.5, 0.75),
            #'text_pipeline__vect__max_features': (None, 7000),
            #'text_pipeline__tfidf__use_idf': (True, False),
            #'clf__estimator__max_depth': [2, 3],
            #'clf__estimator__min_samples_split': [2, 3],
            #'clf__estimator__n_estimators': [50, 70],
            #'clf__estimator__max_leaf_nodes' : [3,4]
        },
    ]

    gscv = GridSearchCV(pipeline, parameters, cv=4, n_jobs=2, return_train_score=False, verbose=3)
    #gscv.fit(train_data, train_labels)
    #print(gscv.best_params_)
    
    return gscv

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [12]:
#def display_results(y_test, y_pred):
#    labels = np.unique(y_pred)
#    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
#    accuracy = (y_pred == y_test).mean()

#    print("Labels:", labels)
#    print("Confusion Matrix:\n", confusion_mat)
#    print("Accuracy:", accuracy)

In [11]:
def train(model_specified):
    
    X, y, column_names = load_data()
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state = 42, shuffle = True)

    model = model_specified

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    #display_results(y_test, y_pred)
    return y_test, y_pred, column_names

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [14]:
y_test, y_pred, column_names = train(model_pipeline())

In [15]:
target_dataframe = pd.DataFrame(y_pred, columns = column_names)

In [16]:
for i, value in enumerate(target_dataframe):
    print("Model Confusion Matrix for the", value, "are below")
    print(classification_report(y_test.iloc[:,i], target_dataframe.iloc[:,i] ))

Model Confusion Matrix for the related are below
             precision    recall  f1-score   support

          0       0.33      0.13      0.19      1861
          1       0.77      0.91      0.83      5951
          2       0.17      0.06      0.09        51

avg / total       0.66      0.72      0.68      7863

Model Confusion Matrix for the request are below
             precision    recall  f1-score   support

          0       0.84      0.97      0.90      6555
          1       0.41      0.09      0.15      1308

avg / total       0.77      0.83      0.78      7863

Model Confusion Matrix for the offer are below
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7831
          1       0.00      0.00      0.00        32

avg / total       0.99      1.00      0.99      7863

Model Confusion Matrix for the aid_related are below
             precision    recall  f1-score   support

          0       0.60      0.81      0.69      46

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

#### Using Parameters Map Used Earlier to estimate the time taken

In [17]:
parameters = {
        #'text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'text_pipeline__vect__max_df': (0.5, 0.75),
        #'text_pipeline__vect__max_features': (None, 7000),
        #'text_pipeline__tfidf__use_idf': (True, False),
        #'mclf__estimator__max_depth': [2, 3],
        #'mclf__estimator__min_samples_split': [2, 3],
        'mclf__estimator__n_estimators': [50, 70],
        #'mclf__estimator__max_leaf_nodes' : [3,4]
        
        
    }
#cv = 

In [18]:
com = 1
for x in parameters.values():
    com *= len(x)
print('There are {} combinations'.format(com))

There are 4 combinations


##### Assuming 100 calculations per second, time taken would be

In [19]:
print('This would take {:.0f} minutes to finish.'.format((100 * com) / (60)))

This would take 7 minutes to finish.


### Predicting with GridSearch

In [20]:
y_test_gs, y_pred_gs, column_names = train(model_pipeline_with_GS())

Fitting 2 folds for each of 2 candidates, totalling 4 fits
[CV] mclf__estimator__n_estimators=50 ................................
[CV] mclf__estimator__n_estimators=50 ................................
[CV] mclf__estimator__n_estimators=70 ................................
[CV] mclf__estimator__n_estimators=70 ................................
[CV]  mclf__estimator__n_estimators=50, score=0.1529655473179241, total=20.1min
[CV]  mclf__estimator__n_estimators=50, score=0.15623637156563455, total=20.3min
[CV]  mclf__estimator__n_estimators=70, score=0.15743567378979503, total=24.8min
[CV]  mclf__estimator__n_estimators=70, score=0.1599433057130397, total=25.1min


[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed: 27.8min finished


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [21]:
target_dataframe_2 = pd.DataFrame(y_pred_gs, columns = column_names)

In [22]:
for i, value in enumerate(target_dataframe_2):
    print("Model Confusion Matrix for the", value, "are below")
    print(classification_report(y_test_gs.iloc[:,i], target_dataframe_2.iloc[:,i] ))

Model Confusion Matrix for the related are below
             precision    recall  f1-score   support

          0       0.33      0.04      0.07      1861
          1       0.76      0.97      0.85      5951
          2       0.06      0.02      0.03        51

avg / total       0.65      0.75      0.66      7863

Model Confusion Matrix for the request are below
             precision    recall  f1-score   support

          0       0.84      0.99      0.91      6555
          1       0.36      0.04      0.07      1308

avg / total       0.76      0.83      0.77      7863

Model Confusion Matrix for the offer are below
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7831
          1       0.00      0.00      0.00        32

avg / total       0.99      1.00      0.99      7863

Model Confusion Matrix for the aid_related are below
             precision    recall  f1-score   support

          0       0.60      0.83      0.70      46

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
y_test_n, y_pred_n, column_names = train(model_pipeline_with_Multiple())

Fitting 4 folds for each of 2 candidates, totalling 8 fits
[CV] clf__estimator=MultiOutputClassifier(estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
           n_jobs=1) 
[CV] clf__estimator=MultiOutputClassifier(estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
           n_jobs=1) 
[CV]  clf__estimator=MultiOutputClassifier(estimator=DecisionTreeClassifier(class_weight=No

In [ ]:
target_dataframe_3 = pd.DataFrame(y_pred_n, columns = column_names)

In [ ]:
for i, value in enumerate(target_dataframe_2):
    print("Model Confusion Matrix for the", value, "are below")
    print(classification_report(y_test_n.iloc[:,i], target_dataframe_3.iloc[:,i] ))

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.